In [ ]:
import gym
import numpy as np
import time, pickle, os
import pandas as pd
from matplotlib import pyplot

env = gym.make('FrozenLake-v1')

epsilon = 0.9
min_epsilon = 0.1
max_epsilon = 1.0
decay_rate = 0.001

total_episodes = 20000
max_steps = 100

lr_rates = [0.048] # np.arange(0.045,0.055, 0.001)
gammas = [0.9850] # np.arange(0.985, 0.986, 0.001)

In [ ]:
def choose_action(state):
    action = 0
    if np.random.uniform(0,1) < epsilon:
        action = env.action_space.sample()
    else:
        action = np.argmax(Q[state,:])
    return action

In [ ]:
def learn(state, state2, reward, action):
    predict = Q[state, action]
    target = reward + gamma * np.max(Q[state2, :])
    Q[state, action] = Q[state, action] + lr_rate* (target - predict)

In [ ]:
# Start

rewards = np.zeros((len(gammas), len(lr_rates), total_episodes))

for gamma_index in np.arange(0,len(gammas)):
    gamma = gammas[gamma_index]
    for lr_index in np.arange(0,len(lr_rates)):
        lr_rate = lr_rates[lr_index]
        
        print("Learning rate: %0.4f and Discount factor gamma: %0.4f" % (lr_rate, gamma))
        Q = np.zeros((env.observation_space.n, env.action_space.n))
        
        for episode in range(total_episodes):
            episode_reward = 0
            state = env.reset()
            t = 0

            while (t < max_steps):
                #env.render()
                action = choose_action(state)
                state2, reward, done, info = env.step(action)
                learn(state, state2, reward, action)

                state = state2

                t += 1
                episode_reward += reward

                if done:
                    break
            epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)   
                #time.sleep(0.1)
            # print("Episode %d with reward %f" % (episode, episode_reward))
            rewards[gamma_index,lr_index,episode] = episode_reward
            
        total_rewards = np.sum(rewards[gamma_index,lr_index,:])
        print("Score over time: ", total_rewards/total_episodes)
        average_reward = total_rewards/total_episodes
        with open("frozenLake_qTable_gamma_%0.4f_lr_rate_%0.4f_reward_%0.4f.pkl" % (gamma,lr_rate,average_reward), 'wb') as f:
            pickle.dump(Q, f)
            
with open("frozenLake_qTable_rewards.pkl",'wb') as f:
    pickle.dump(rewards, f)

In [ ]:
# %matplotlib notebook

d = pd.Series(rewards[0,0,:])
pyplot.plot(np.arange(0,total_episodes),d.rolling(100).mean())
pyplot.show()

In [ ]:
rewards